# CYP21A2
to do

In [1]:
import gpsea
import hpotk
from IPython.display import display, HTML

store = hpotk.configure_ontology_store()
hpo = store.load_minimal_hpo()
print(f'Loaded HPO v{hpo.version}')
print(f"Using gpsea version {gpsea.__version__}")

Loaded HPO v2024-08-13
Using gpsea version 0.4.1.dev0


In [2]:
from ppktstore.registry import configure_phenopacket_registry
from gpsea.preprocessing import configure_caching_cohort_creator

cohort_name = "CYP21A2"	


cohort_creator = configure_caching_cohort_creator(hpo)
from gpsea.preprocessing import load_phenopacket_folder
pp_dir = '/Users/robin/GIT/phenopacket-store/notebooks/CYP21A2/phenopackets/'
cohort, qc_results = load_phenopacket_folder(pp_dir, cohort_creator)  



Individuals Processed: 11individuals [00:39,  3.57s/individuals]


ValueError: Unknown structural type SO:0001636

In [8]:
from pyphetools.visualization import PhenopacketIngestor
pp_dir = '/Users/robin/GIT/phenopacket-store/notebooks/CYP21A2/phenopackets'
ingestor = PhenopacketIngestor(indir=pp_dir)
ppkt_lst = ingestor.get_phenopacket_list()


[pyphetools] Ingested 69 GA4GH phenopackets.


In [15]:
pplist = list()
for ppkt in ppkt_lst:
    gi = ppkt.interpretations[0].diagnosis.genomic_interpretations[0]
    vi = gi.variant_interpretation
    vd = vi.variation_descriptor
    if vd.HasField("structural_type"):
        continue
    pplist.append(ppkt)



In [16]:
from gpsea.preprocessing import load_phenopackets
cohort, qc_results = load_phenopackets(pplist, cohort_creator)  

Individuals Processed: 100%|██████████| 67/67 [01:03<00:00,  1.06individuals/s]


In [17]:
qc_results.summarize()

Validated under none policy
Showing errors and warnings
Phenopackets
  patient #24
    phenotype-features
     warnings:
     ·No phenotype terms were left after the validation. Revise the phenotype terms and try again


In [18]:
pplist[24]

id: "PMID_30968594_individual_29"
subject {
  id: "individual 29"
  sex: MALE
}
measurements {
  assay {
    id: "LOINC:1668-3"
    label: "17-Hydroxyprogesterone[Mass/Vol]"
  }
  value {
    quantity {
      unit {
        id: "UCUM:ng/dL"
        label: "nanogram per deciliter"
      }
      value: 238.0
    }
  }
}
interpretations {
  id: "individual 29"
  progress_status: SOLVED
  diagnosis {
    disease {
      id: "OMIM:201910"
      label: "Adrenal hyperplasia, congenital, due to 21-hydroxylase deficiency"
    }
    genomic_interpretations {
      subject_or_biosample_id: "individual 29"
      interpretation_status: CAUSATIVE
      variant_interpretation {
        variation_descriptor {
          id: "var_QnpUKQDkvkXSkGfiQThrqIRvE"
          gene_context {
            value_id: "HGNC:2600"
            symbol: "CYP21A2"
          }
          expressions {
            syntax: "hgvs.c"
            value: "NM_000500.9:c.293-13C>G"
          }
          expressions {
            synt

# "id": "LOINC:2986-8",
        "label": "Testosterone[Mass/Vol]"

#  

In [ ]:
testor_predicate = from_loinc()